In [71]:
import pandas as pd
import requests
from datetime import datetime, timedelta
from constants.definitions import API_KEY, ACCOUNT_ID, OANDA_URL
import pytz

In [59]:
# initialise a session and set headers
session = requests.Session()
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
})

params = {
    'granularity': "S5",
    'count': 5,
    'price': 'BAM',
    'from' : datetime(2020, 1, 1).isoformat() + 'Z',
}

In [60]:
params['from']

'2020-01-01T00:00:00Z'

In [61]:
response = session.get(f"{OANDA_URL}/instruments/GBP_JPY/candles", params=params)

In [62]:
if "candles" in response.json():
    print("Tri")

Tri


In [64]:
df = pd.DataFrame(response.json())
# Extract and format the relevant columns
# df['time'] = pd.to_datetime(df['time'])
# df['open'] = df['mid'].apply(lambda x: x['o'])
# df['high'] = df['mid'].apply(lambda x: x['h'])
# df['low'] = df['mid'].apply(lambda x: x['l'])
# df['close'] = df['mid'].apply(lambda x: x['c'])
# df = df[['volume', 'time', 'open', 'high', 'low', 'close']]

df

,instrument,granularity,candles
0,GBP_JPY,S5,"{'complete': True, 'volume': 1, 'time': '2020-..."
1,GBP_JPY,S5,"{'complete': True, 'volume': 2, 'time': '2020-..."
2,GBP_JPY,S5,"{'complete': True, 'volume': 2, 'time': '2020-..."
3,GBP_JPY,S5,"{'complete': True, 'volume': 2, 'time': '2020-..."
4,GBP_JPY,S5,"{'complete': True, 'volume': 3, 'time': '2020-..."


In [83]:
def fetch_candles(instrument: str, start_date: str, end_date: str, granularity: str, oanda_url: str, api_key: str) -> pd.DataFrame:
    # session = requests.Session()
    session.headers.update({
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    })

    start_date = datetime.fromisoformat(start_date).replace(tzinfo=pytz.UTC)
    end_date = datetime.fromisoformat(end_date).replace(tzinfo=pytz.UTC)
    
    url = f"{oanda_url}/instruments/{instrument}/candles"
    all_candles = []
    current_date = start_date
    
    print(f"Starting to fetch candles from {start_date} to {end_date} of {instrument} with granularity {granularity}")
    
    while current_date < end_date:
        print(f"Now fetching candles for {current_date}")
        params = {
            'granularity': granularity,
            'count': 5000,
            'price': 'BAM',
            'from' : current_date.isoformat(),
        }
        
        res = session.get(url, params=params)
        if res.status_code != 200:
            print(f"Error in fetch_candles(): {res.status_code}")
        if not "candles" in res.json():
            print(f"Error in fetch_candles(): response object has no 'candles' key. Current date: {current_date}")
        else:
            data = res.json()['candles']
            all_candles.extend(data)
            last_candle_time = pd.to_datetime(data[-1]['time'])
            current_date = last_candle_time + timedelta(seconds=1)
            print(f"Finished fetching candles for {current_date}")
        
    else:
        return pd.DataFrame(all_candles)

In [84]:
candles_df = fetch_candles(
    instrument='GBP_JPY',
    start_date="2020-01-01",
    end_date="2020-01-02",
    granularity='H1',
    oanda_url=OANDA_URL,
    api_key=API_KEY
)

Starting to fetch candles from 2020-01-01 00:00:00+00:00 to 2020-01-02 00:00:00+00:00 of GBP_JPY with granularity H1
Now fetching candles for 2020-01-01 00:00:00+00:00
Finished fetching candles for 2020-01-02 02:00:01+00:00


In [82]:
candles_df.time.max()

'2020-10-20T05:00:00.000000000Z'

In [113]:
date1 = datetime.fromisoformat("2020-01-01").replace(tzinfo=pytz.UTC)
date2 = datetime.fromisoformat("2020-01-02").replace(tzinfo=pytz.UTC)

In [114]:
diff=date2 - date1

In [115]:
diff.total_seconds() / 5

17280.0

In [120]:
params = {
    'granularity': 'H1',
    'from': date1.isoformat(),
    'to': date2.isoformat(),
    'price': 'B',
}

In [121]:
response = session.get(f"{OANDA_URL}/instruments/GBP_JPY/candles", params=params)

In [123]:
len(response.json())

3

In [124]:
df = pd.DataFrame(response.json()["candles"])
df

,complete,volume,time,bid
0,True,874,2020-01-01T22:00:00.000000000Z,"{'o': '144.032', 'h': '144.137', 'l': '143.769..."
1,True,807,2020-01-01T23:00:00.000000000Z,"{'o': '143.965', 'h': '144.073', 'l': '143.880..."
